# MyoSDK Tutorial

## What is MyoSDK?

MyoSapiens SDK (MyoSDK) is a Python library that allows you to **Retarget motion** from marker datas onto a 3D character.

## What You'll Learn

This tutorial will walk you through the complete process:
1. Setting up the MyoSDK client
2. Apply marker data describing a motion to a 3D character (retargeting)
3. Downloading the final animated character file

## Prerequisites

Before starting, make sure you have:
- **Python 3.8 or higher** installed
- **An API key** from MyoLab (you'll need to sign up and get your API key)
- **A Mocap file in c3d format** showing a person moving
- **A Markerset file ** describing approximately where the markers are placed on the body
- **Basic Python knowledge** (understanding variables, functions, and basic syntax)

## Installation

If you haven't installed `myosdk` yet, run this command in your terminal:
```bash
pip install myosdk
```

In [ ]:
# Step 1: Import required libraries
import os
import myosdk
from myosdk import Client
# Step 2: Set up your API key
# The API key is like a password that authenticates you with the MyoSDK service.
# You can get your API key from your MyoLab account dashboard at dev.myolab.ai.
# Instructions are available at https://docs.myolab.ai/docs/myosdk/getting-started/api-key

# Option 1 (Recommended): Set it as an environment variable
# In your terminal, run: export MYOSDK_API_KEY="your-api-key-here"
# Or on Windows: set MYOSDK_API_KEY=your-api-key-here

# Option 2: Replace "..." below with your actual API key (less secure, but easier for testing)
api_key = os.getenv("MYOSDK_API_KEY", "...")

# Step 3: Create the client
# The client is your connection to the MyoSDK service. It handles all communication
# with the servers. The base_url tells it which server to connect to.
client = Client(api_key=api_key, base_url="https://v2m-alb-us-east-1.myolab.ai")

# Verify the connection worked
print("Client initialized successfully.")
print(f"✓ You're ready to use MyoSDK version {myosdk.__version__}!")

## 1. Upload Input Assets

We need to upload a c3d and markerset files to the platform. These will be used to...

In [ ]:
c3d_path = "../data/35_30.c3d"
markerset_path = "../markersets/cmu_markerset.xml"

# Upload C3D motion capture file
c3d_asset = client.assets.upload_file(c3d_path)
print(c3d_asset)
print("<<< TODO: add validation in sdk e.g. client.validate_mocap_data(c3d_path)")

# Upload markerset XML file
markerset_asset = client.assets.upload_file(markerset_path)
print(markerset_asset)
print("<<< TODO: add validation in sdk e.g. client.validate_markerset(markerset_path)")


## 2. Motion Retargeting

Use the extracted motion data (trackers) to retarget onto a 3D character. This is called "retargeting."

**What is retargeting?** Think of it like this: you recorded someone dancing, extracted their dance moves, and now you're applying those exact same moves to a 3D character. The character will move in the same way as the person in your video.

**What you need:**
- The `c3d_asset` describing the motion data
- The `markerset_asset` describing the association of motion data to body parts


In [ ]:
# Start retargeting job
job = client.jobs.start_retarget(
    c3d_asset_id=c3d_asset["asset_id"],
    markerset_asset_id=markerset_asset["asset_id"],
)

# Wait for job to complete
result = client.jobs.wait(job["job_id"])
print("<<< TODO: for the wait add some progress bar / spinner / info update")
retarget_output_asset_id = result["output"]["retarget_output_asset_id"]
print(retarget_output_asset_id)

## 3. Download Result

Finally, we'll download the joint angles.

**Note:** Make sure the output directory exists, or the download will create it for you.

In [ ]:
output_npy_path = "tmp-assets/tutorial_output.npz"

print(f"Downloading result to {output_npy_path}...")
os.makedirs(os.path.dirname(output_npy_path), exist_ok=True)
client.assets.download(retarget_output_asset_id, output_npy_path)

print("Download complete!")
print("<<< TODO: can we return the file or need to write it to disk?")

In [ ]:
import numpy as np
import pandas as pd
data = np.load(output_npy_path)
joint_angles = data['joint_angles_degrees'].squeeze()
joint_names = data['joint_names']
df = pd.DataFrame(joint_angles, columns=[jn for jn in joint_names])
df.insert(0, "frame", df.index)
df.head()